# メトロポリス・ヘイスティング法を用いたモンテカルロ最尤推定
パラメーターなど

In [49]:
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [35]:
n = 1000
σ = 1.5
μ = 10
x = σ*np.random.randn(n) + μ

gaussian = np.frompyfunc(lambda x,σ,μ: 1.0/np.sqrt(2*np.pi*σ**2)*np.exp(-((x-μ)/σ)**2/2.0),3,1)
likelihood = lambda p: np.log(p).sum()

In [37]:
σ_series = [1.0]
μ_series = [0.0]

p = gaussian(x,σ_series[-1],μ_series[-1]).astype(np.double)
l = likelihood(p)
likelihood_series = [l]



for i in range(1000):
    new_σ = σ_series[-1] + np.random.randn()*0.1
    new_μ = μ_series[-1] + np.random.randn()*0.1

    p = gaussian(x,new_σ,new_μ).astype(np.double)
    new_l = likelihood(p)

    alpha = np.exp(new_l - likelihood_series[-1])

    if alpha < np.random.uniform():
        new_σ = σ_series[-1]
        new_μ = μ_series[-1]
        new_l = likelihood_series[-1]
    
    σ_series.append(new_σ)
    μ_series.append(new_μ)
    likelihood_series.append(new_l)

C:\Users\Ruofan\AppData\Local\Temp\ipykernel_21068\768423018.py:17: RuntimeWarning: overflow encountered in exp
  alpha = np.exp(new_l - likelihood_series[-1])


In [54]:
fig = go.Figure()
go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
fig.add_trace(go_element(None, σ_series , 'σ'))
fig.add_trace(go_element(None, μ_series , 'μ'))
fig.update_layout(xaxis_title='iter', yaxis_title='value')
fig.show()

In [88]:
fig = go.Figure()
go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
fig.add_trace(go.Histogram(x=x,histnorm='probability density',name='hist'))
x_range = np.linspace(np.min(x),np.max(x), 1000)
fig.add_trace(go_element(x_range , gaussian(x_range,σ_series[-1], μ_series[-1]) , 'density func'))
fig.update_layout(xaxis_title='x', yaxis_title='prob')
fig.show()

In [107]:
import plotly.graph_objects as go
import numpy as np

# Create figure
fig = go.Figure()

go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
x_range = np.linspace(np.min(x)-10,np.max(x), 1000)


display_iter = range(len(σ_series))[::20]


# Add traces, one for each slider step
for step in display_iter:
    fig.add_trace(
        go.Scatter(
            visible=False,
            mode='lines',
            name='density',
            x=x_range,
            y=gaussian(x_range,σ_series[step], μ_series[step])
        )
    )

fig.add_trace(go.Histogram(x=x,histnorm='probability density',name='hist'))

# Make 10th trace visible
fig.data[0].visible = True
fig.data[-1].visible = True

# Create and add slider
steps = []
for i,j in enumerate(display_iter):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}],
        label=str(j)
    )
    step["args"][0]["visible"][i] = True
    step["args"][0]["visible"][-1] = True
    steps.append(step)


sliders = [dict(
    active=0,
    currentvalue={"prefix": "iteration: "},
    steps=steps
)]


fig.update_layout(
    sliders=sliders
)

fig.show()